In [1]:
import torch
print(torch.__version__)      # e.g., '2.3.0+cu121' means PyTorch 2.3.0 built with CUDA 12.1
print(torch.version.cuda)

import torch
import flash_attn
print("Flash Attention installed successfully!")
print(flash_attn.__version__)

2.7.1+cu126
12.6


ModuleNotFoundError: No module named 'flash_attn'

In [26]:
from transformers import Wav2Vec2BertConfig, Wav2Vec2BertModel, Wav2Vec2BertProcessor

# Initializing a Wav2Vec2Bert facebook/wav2vec2-bert-rel-pos-large style configuration
configuration = Wav2Vec2BertConfig()

# Initializing a model (with random weights) from the facebook/wav2vec2-bert-rel-pos-large style configuration
model = Wav2Vec2BertModel(configuration)

processor = Wav2Vec2BertProcessor.from_pretrained("facebook/w2v-bert-2.0")

# Accessing the model configuration
configuration = model.config


TypeError: expected str, bytes or os.PathLike object, not NoneType

START HERE

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from transformers import Wav2Vec2BertProcessor, Wav2Vec2BertForCTC
import torch
import torchaudio
import os
import pandas as pd

# Choose a pre-trained model. Popular choices include:
# "facebook/wav2vec2-base-960h"
# "facebook/wav2vec2-large-960h"
# "facebook/wav2vec2-large-xlsr-53" (for multilingual models)
model_name = "facebook/wav2vec2-base-960h"

# Load the processor (tokenizer and feature extractor)
processor = Wav2Vec2Processor.from_pretrained(model_name)

# Load the model (Wav2Vec2ForCTC is commonly used for ASR)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# This is standard
target_sample_rate = 16000

# Move to gpu if avail
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Train only the classification layer
model.freeze_feature_encoder()

print(f"\n(!) Loaded {model_name} on {device}")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



(!) Loaded facebook/wav2vec2-base-960h on cuda


Do a sample evaluation


In [8]:
def process_and_transcribe(audio_path, target_sample_rate=16000):
    speech_array, sampling_rate = torchaudio.load(audio_path)

    # Turn into mono
    if speech_array.ndim > 1:
        speech_array = speech_array.mean(dim=0, keepdim=True) 

    # Remove outer dim of speech array
    speech_array = speech_array.squeeze(0) 

    if sampling_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=target_sample_rate)
        speech_array = resampler(speech_array)

    input_features = processor(
        speech_array.numpy(), # Wav2Vec2 models typically expect numpy arrays
        sampling_rate=target_sample_rate,
        return_tensors="pt"
    ).input_values.to(device)

    with torch.no_grad(): # Disable gradient calculations to save memory and speed up inference
        logits = model(input_features).logits
        
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription.lower()

audio_path = "C:\\Users\\dacla\\Documents\\auto-censoring-local\\separated\\mdx_extra\\clint\\vocals.mp3"
process_and_transcribe(audio_path)

'<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>'

Another trial with a DALI song



In [ ]:
from pydub import AudioSegment
import demucs.separate

def extract_portions(input_audio_path, times):
    # Get path names for inputs
    root, ext = os.path.splitext(os.path.basename(input_audio_path))
    dir = os.path.dirname(input_audio_path)

    # Define path names for demucs
    demucs_path = os.path.abspath(f"separated/mdx_extra/{root}")
    vocals_path = os.path.join(demucs_path, "vocals.wav")
    no_vocals_path = os.path.join(demucs_path, "no_vocals.wav")

    # Do not rerun if not needed
    if not os.path.isfile(vocals_path):
        demucs.separate.main(["--two-stems", "vocals", "-n", "mdx_extra", input_audio_path])

    audio = AudioSegment.from_file(vocals_path)

    segment_names = []

    for i in range(len(times)):
        start = times[i][0] * 1000
        end = times[i][1] * 1000

        # Split the portion that contains vocals
        portion = audio[start:end]

        # Keep in order
        output_path = os.path.join(dir, f"{root}-chunk-{i}.mp3")
        portion.export(output_path, format="mp3")

        segment_names.append(output_path)
    
    # Delete splits
    os.remove(vocals_path)
    os.remove(no_vocals_path)
    os.rmdir(demucs_path)

    return segment_names

In [13]:
dali_dir = os.path.abspath(f"../DALI-audio")

for id in os.listdir(dali_dir)[:10]:
    for file in os.listdir(os.path.join(dali_dir, id)):
        root, ext = os.path.splitext(file)

        if ext == '.mp3':
            audio_path = os.path.join(os.path.join(dali_dir, id), file)

    df = pd.read_csv(f"../DALI-audio/{id}/lyrics-paragraphs.csv")

    times = []
    for row in df.itertuples():
        start = row[2]
        end = row[3]
        times.append([start, end])

    extract_portions(audio_path, times)

Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in C:\Users\dacla\Documents\auto-censoring-local\separated\mdx_extra
Separating track c:\Users\dacla\Documents\DALI-audio\001940b614eb43f4a0c826d49a67d66d\woman-in-love.mp3


KeyboardInterrupt: 

In [4]:
dali_dir = os.path.abspath(f"../DALI-audio")

# Just select the first song
id = os.listdir(dali_dir)[2]
print(id)

for file in os.listdir(os.path.join(dali_dir, id)):
    root, ext = os.path.splitext(file)

    if ext == '.mp3':
        audio_path = os.path.join(os.path.join(dali_dir, id), file)

df = pd.read_csv(f"../DALI-audio/{id}/lyrics-paragraphs.csv")

times = []
for row in df.itertuples():
    start = row[2]
    end = row[3]
    times.append([start, end])

segment_names = extract_portions(audio_path, times)
lyrics_true = df['words'].tolist()

007c0152242340008ff45781a9b08546
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in C:\Users\dacla\Documents\auto-censoring-local\separated\mdx_extra
Separating track c:\Users\dacla\Documents\DALI-audio\007c0152242340008ff45781a9b08546\wildfire.mp3


100%|███████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:02<00:00, 102.45seconds/s]
100%|███████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:02<00:00, 100.21seconds/s]
100%|███████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:02<00:00, 114.96seconds/s]
100%|███████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:01<00:00, 115.84seconds/s]


In [11]:
from evaluate import load

n = len(lyrics_true)
lyrics_pred = []
for i in range(n):
    chunk_path = segment_names[i]
    pred = process_and_transcribe(audio_path=chunk_path)
    lyrics_pred.append(pred.lower())

# Word Error Rate metric
wer_metric = load("wer")

score = wer_metric.compute(predictions=lyrics_pred, references=lyrics_true)

print(lyrics_pred)
print(f"WER score: {score}")

['<unk><unk>', '<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>', '<unk><unk><unk><unk><unk><unk>', '<unk><unk><unk><unk>', '<unk><unk><unk><unk><unk>', '<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>', '<unk><unk><unk><unk><unk><unk><unk><unk><unk>', '<unk><unk><unk><unk><unk>', '<unk><unk><unk><unk><unk><unk><unk><unk>'

Now for training

In [ ]:
# Tokenize the true lyrics
lyrics_true_tok = processor.tokenizer(lyrics_true, padding=True)['input_ids']
y = torch.tensor(lyrics_true_tok).to(device)

# y is the labels


tensor([3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3,
        3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 4, 3,
        3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [6]:
from torch.nn.utils.rnn import pad_sequence

# process audio
def process_audio(audio_path, target_sample_rate=16000):
    speech_array, sampling_rate = torchaudio.load(audio_path)

    # Turn into mono
    if speech_array.ndim > 1:
        speech_array = speech_array.mean(dim=0, keepdim=True) 

    # Remove outer dim of speech array
    speech_array = speech_array.squeeze(0) 

    if sampling_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=target_sample_rate)
        speech_array = resampler(speech_array)

    input_features = processor(
        speech_array.numpy(), # Wav2Vec2 models typically expect numpy arrays
        sampling_rate=target_sample_rate,
        return_tensors="pt"
    ).input_values.to(device)

    return input_features


processed_segments = []

for audio_path in segment_names:
    proc = process_audio(audio_path)
    processed_segments.append(proc.squeeze(0))

# X is the input to our training loop
X = pad_sequence(processed_segments, batch_first=True)

X.shape

torch.Size([9, 471761])

Training loop

In [10]:
import torch.optim

optimizer = torch.optim.AdamW(model.parameters(), lr=.001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.1)

model.train()

for epoch in range(20):
    outputs = model(X, labels=y)  
    loss = outputs.loss

    loss.backward()
    optimizer.step()
    #scheduler.step()
    optimizer.zero_grad()

    print(f'Epoch {epoch + 1}: loss {loss.item():.4f}')

Epoch 1: loss 1229.5851
Epoch 2: loss 16281.6602
Epoch 3: loss 7500.6543
Epoch 4: loss 13785.4355
Epoch 5: loss 4012.8577
Epoch 6: loss 5635.2363
Epoch 7: loss 478.2917
Epoch 8: loss 826.8695
Epoch 9: loss 14415.0410
Epoch 10: loss 3383.9194
Epoch 11: loss 5849.9966
Epoch 12: loss 11001.1699
Epoch 13: loss 7768.5981
Epoch 14: loss 3361.2402
Epoch 15: loss 6864.5527
Epoch 16: loss 2723.0771
Epoch 17: loss 1344.1112
Epoch 18: loss -76.8533
Epoch 19: loss 1971.2827
Epoch 20: loss 974.2751
